In [1]:
%pip -q install google-genai

In [16]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [17]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [18]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [19]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [20]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [21]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [30]:
def agente_coletor_de_dados(nome_cliente, produto, objetivo):
    coletor = Agent(
        name="agente_coletor_de_dados",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um agente comercial que prepara propostas de vendas.
        Receba as seguintes informações:
        - Nome do cliente
        - Produto ou serviço a ser oferecido
        - Objetivo da proposta (ex: fechar parceria, realizar venda, agendar reunião)

        Sua tarefa é levantar os principais pontos que devem constar na proposta com base nesses dados:
        • Benefícios para o cliente
        • Características relevantes do produto
        • Justificativas comerciais que apoiam o envio dessa proposta.
        """,
        description="Agente que prepara informações comerciais básicas para proposta"
    )

    entrada = f"Cliente: {nome_cliente}\nProduto: {produto}\nObjetivo: {objetivo}"
    dados_preparados = call_agent(coletor, entrada)
    return dados_preparados


In [31]:
def agente_estrategista(dados_comerciais):
    estrategista = Agent(
        name="agente_estrategista",
        model="gemini-2.0-flash",
        instruction="""
        Você é um estrategista comercial responsável por montar a estrutura lógica de uma proposta de vendas.
        Com base nas informações recebidas (cliente, produto, benefícios e justificativas), elabore uma estrutura clara com os seguintes tópicos:

        1. Introdução personalizada para o cliente
        2. Apresentação do produto ou serviço
        3. Benefícios diretos para o cliente
        4. Justificativas comerciais
        5. Proposta de ação (call to action: reunião, contato, assinatura etc.)

        Retorne isso em formato de tópicos para ser usado por um redator.
        """,
        description="Agente que define a estrutura da proposta comercial"
    )

    entrada_estrategica = f"Informações comerciais: {dados_comerciais}"
    estrutura_da_proposta = call_agent(estrategista, entrada_estrategica)
    return estrutura_da_proposta


In [35]:
def agente_gerador_de_proposta(estrutura):
    redator = Agent(
        name="agente_gerador_de_proposta",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator comercial. Sua tarefa é transformar uma estrutura de proposta de vendas em um texto claro, profissional e persuasivo.

        O tom deve ser formal e direto, mas amigável. Use frases simples e parágrafos curtos.
        Ao final, inclua uma chamada para ação (ex: “Aguardamos seu retorno para dar seguimento à parceria.”).
        """,
        description="Agente que redige a proposta comercial"
    )

    entrada_redator = f"Estrutura da proposta: {estrutura}"
    proposta_gerada = call_agent(redator, entrada_redator)
    return proposta_gerada


In [36]:
def agente_revisor_de_proposta(proposta):
    revisor = Agent(
        name="agente_revisor_de_proposta",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor profissional de textos comerciais.
        Revise o texto da proposta abaixo observando:
        - Clareza
        - Gramática e ortografia
        - Tom adequado para negócios
        - Sugestões de melhoria, se necessário

        Se o texto estiver bom, diga: 'A proposta está pronta para envio.' Caso contrário, aponte melhorias.
        """,
        description="Agente que revisa proposta comercial"
    )

    entrada_revisor = f"Proposta: {proposta}"
    texto_final = call_agent(revisor, entrada_revisor)
    return texto_final


In [37]:
print("🚀 Iniciando o Gerador de Propostas Comerciais com 4 Agentes Autônomos 🚀")

# --- Entrada do usuário ---
nome_cliente = input("📌 Nome do cliente: ").strip()
produto = input("📦 Produto ou serviço oferecido: ").strip()
objetivo = input("🎯 Objetivo da proposta (ex: agendar reunião, fechar parceria, enviar orçamento): ").strip()

if not nome_cliente or not produto or not objetivo:
    print("⚠️ Todos os campos são obrigatórios.")
else:
    print("\n✅ Iniciando geração da proposta...\n")

    dados = agente_coletor_de_dados(nome_cliente, produto, objetivo)
    print("\n--- 🧠 Resultado do Agente 1: Coletor de Dados ---\n")
    display(to_markdown(dados))
    print("--------------------------------------------------------------")

    estrutura = agente_estrategista(dados)
    print("\n--- 🧱 Resultado do Agente 2: Estrategista ---\n")
    display(to_markdown(estrutura))
    print("--------------------------------------------------------------")

    proposta = agente_gerador_de_proposta(estrutura)
    print("\n--- ✍️ Resultado do Agente 3: Gerador de Proposta ---\n")
    display(to_markdown(proposta))
    print("--------------------------------------------------------------")

    proposta_final = agente_revisor_de_proposta(proposta)
    print("\n--- ✅ Resultado do Agente 4: Revisor ---\n")
    display(to_markdown(proposta_final))
    print("--------------------------------------------------------------")


🚀 Iniciando o Gerador de Propostas Comerciais com 4 Agentes Autônomos 🚀
📌 Nome do cliente: Braspress
📦 Produto ou serviço oferecido: eletrodomesticos
🎯 Objetivo da proposta (ex: agendar reunião, fechar parceria, enviar orçamento): enviar orçamento

✅ Iniciando geração da proposta...


--- 🧠 Resultado do Agente 1: Coletor de Dados ---



> ## Proposta de Orçamento - Braspress
> 
> **Principais Pontos a serem Destacados:**
> 
> **1. Benefícios para a Braspress:**
> 
> *   **Qualidade e Durabilidade:** Eletrodomésticos de alta qualidade, reduzindo custos com manutenção e reposição a longo prazo.
> *   **Eficiência Energética:** Modelos com alta eficiência energética, diminuindo o consumo de energia e os custos operacionais da Braspress.
> *   **Confiabilidade:** Produtos de marcas renomadas, garantindo a satisfação dos usuários e a reputação da Braspress.
> *   **Suporte Técnico:** Ampla rede de suporte técnico para garantir o funcionamento contínuo dos equipamentos.
> *   **Condições Especiais:** Ofertas e descontos exclusivos para clientes corporativos como a Braspress.
> 
> **2. Características Relevantes dos Produtos:**
> 
> *   **Especificações Técnicas:** Detalhamento das especificações técnicas de cada eletrodoméstico, como capacidade, dimensões, voltagem, etc.
> *   **Modelos:** Apresentar variedade de modelos.
> *   **Certificações e Normas:** Conformidade com as normas de segurança e qualidade, garantindo a segurança dos usuários.
> *   **Design e Ergonomia:** Produtos com design moderno e ergonômico, proporcionando conforto e praticidade no uso.
> *   **Garantia:** Prazos e condições de garantia de cada produto.
> 
> **3. Justificativas Comerciais:**
> 
> *   **Necessidade:** A Braspress necessita de eletrodomésticos para equipar suas instalações, refeitórios, áreas de descanso, etc.
> *   **Oportunidade:** Apresentar um orçamento competitivo e personalizado para atender às necessidades específicas da Braspress.
> *   **Reputação:** Destacar a experiência e a credibilidade da empresa fornecedora no mercado de eletrodomésticos.
> *   **Relacionamento:** Enfatizar o interesse em construir um relacionamento de longo prazo com a Braspress, oferecendo soluções completas e personalizadas.
> *   **Disponibilidade:** Garantir a disponibilidade dos produtos e prazos de entrega rápidos e eficientes.


--------------------------------------------------------------

--- 🧱 Resultado do Agente 2: Estrategista ---



> Ok, aqui está a estrutura lógica da proposta de vendas para a Braspress, com base nas informações fornecidas:
> 
> **1. Introdução Personalizada para a Braspress**
> 
> *   Mencionar a Braspress e sua importância no setor de atuação.
> *   Reconhecer a necessidade da Braspress em equipar suas instalações com eletrodomésticos de qualidade.
> *   Expressar o entusiasmo em apresentar uma proposta de orçamento competitiva e personalizada.
> 
> **2. Apresentação do Produto ou Serviço**
> 
> *   Apresentar a empresa fornecedora de eletrodomésticos, destacando sua experiência e credibilidade no mercado.
> *   Detalhar as características relevantes dos produtos oferecidos:
>     *   Especificações Técnicas (capacidade, dimensões, voltagem, etc.)
>     *   Variedade de Modelos
>     *   Certificações e Normas de segurança e qualidade
>     *   Design e Ergonomia
>     *   Prazos e condições de Garantia
> 
> **3. Benefícios Diretos para a Braspress**
> 
> *   Qualidade e Durabilidade dos eletrodomésticos, reduzindo custos com manutenção e reposição.
> *   Eficiência Energética dos modelos, diminuindo o consumo de energia e os custos operacionais.
> *   Confiabilidade dos produtos de marcas renomadas, garantindo a satisfação dos usuários e a reputação da Braspress.
> *   Ampla rede de Suporte Técnico para garantir o funcionamento contínuo dos equipamentos.
> *   Condições Especiais: Ofertas e descontos exclusivos para clientes corporativos como a Braspress.
> 
> **4. Justificativas Comerciais**
> 
> *   Reforçar a necessidade da Braspress em equipar suas instalações com eletrodomésticos.
> *   Apresentar o orçamento competitivo e personalizado como uma oportunidade para atender às necessidades específicas da Braspress.
> *   Destacar a reputação da empresa fornecedora no mercado de eletrodomésticos.
> *   Enfatizar o interesse em construir um relacionamento de longo prazo, oferecendo soluções completas e personalizadas.
> *   Garantir a disponibilidade dos produtos e prazos de entrega rápidos e eficientes.
> 
> **5. Proposta de Ação (Call to Action)**
> 
> *   Agendar uma reunião para discutir a proposta em detalhes e responder a eventuais dúvidas.
> *   Disponibilizar um canal de contato direto para esclarecimentos adicionais.
> *   Convidar a Braspress a formalizar o aceite da proposta para dar início ao processo de fornecimento.


--------------------------------------------------------------

--- ✍️ Resultado do Agente 3: Gerador de Proposta ---



> **Prezada equipe Braspress,**
> 
> É com grande entusiasmo que apresentamos esta proposta comercial, elaborada sob medida para atender às necessidades da Braspress, empresa de referência no setor em que atua.
> 
> **Apresentação**
> 
> Somos uma empresa especializada no fornecimento de eletrodomésticos, com ampla experiência e credibilidade no mercado. Nossa proposta inclui produtos com as seguintes características:
> 
> *   Especificações Técnicas (capacidade, dimensões, voltagem, etc.)
> *   Variedade de Modelos
> *   Certificações e Normas de segurança e qualidade
> *   Design e Ergonomia
> *   Prazos e condições de Garantia
> 
> **Benefícios para a Braspress**
> 
> Ao escolher nossos eletrodomésticos, a Braspress terá acesso a:
> 
> *   Qualidade e durabilidade, reduzindo custos de manutenção.
> *   Eficiência energética, diminuindo o consumo de energia.
> *   Confiabilidade de marcas renomadas, garantindo a satisfação dos usuários.
> *   Suporte técnico abrangente para garantir o funcionamento contínuo dos equipamentos.
> *   Condições especiais e descontos exclusivos para clientes corporativos.
> 
> **Justificativas Comerciais**
> 
> Nosso objetivo é construir um relacionamento de longo prazo, oferecendo soluções completas e personalizadas. Garantimos a disponibilidade dos produtos e prazos de entrega rápidos e eficientes.
> 
> **Próximos Passos**
> 
> Aguardamos seu retorno para agendarmos uma reunião, discutir a proposta em detalhes e responder a eventuais dúvidas.
> 
> 


--------------------------------------------------------------

--- ✅ Resultado do Agente 4: Revisor ---



> A proposta precisa de alguns ajustes para otimizar a clareza e o impacto:
> 
> **Sugestões:**
> 
> *   **Remover a repetição:** Evite repetir informações, como em "empresa de referência no setor em que atua".
> *   **Ser mais específico:** Detalhe como os benefícios se traduzem em resultados concretos para a Braspress.
> *   **Adicionar valor:** Inclua informações sobre a sua empresa que a diferenciem da concorrência.
> *   **Revisar a linguagem:** A linguagem pode ser mais direta e persuasiva.
> 
> **Proposta revisada:**
> 
> **Prezada equipe Braspress,**
> 
> Apresentamos nossa proposta comercial, criada para atender às necessidades da Braspress.
> 
> **Apresentação**
> 
> Somos especializados no fornecimento de eletrodomésticos, com experiência comprovada e reconhecimento no mercado. Nossa proposta inclui produtos com:
> 
> *   Especificações Técnicas: (capacidade, dimensões, voltagem)
> *   Variedade de Modelos
> *   Certificações de segurança e qualidade
> *   Design ergonômico
> *   Garantia
> 
> \[*Adicionar um breve parágrafo sobre a sua empresa, destacando seus diferenciais e valores.*]
> 
> **Benefícios para a Braspress**
> 
> Ao escolher nossos eletrodomésticos, a Braspress terá:
> 
> *   Redução de custos de manutenção devido à qualidade e durabilidade.
> *   Menor consumo de energia e otimização da eficiência energética.
> *   Satisfação dos usuários com marcas renomadas.
> *   Suporte técnico para garantir o funcionamento dos equipamentos.
> *   Condições especiais e descontos exclusivos.
> 
> **Justificativas Comerciais**
> 
> Nosso objetivo é construir um relacionamento de longo prazo, oferecendo soluções completas. Garantimos a disponibilidade dos produtos e entregas rápidas.
> 
> **Próximos Passos**
> 
> Aguardamos seu retorno para agendarmos uma reunião, discutir a proposta e responder a dúvidas.


--------------------------------------------------------------
